In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd

PATH = ("/content/drive/MyDrive/Spotify_Project/4_Feature_Engineering/spotify_with_features.pkl")

with open(PATH, "rb") as f:
    DF = pickle.load(f)

print("Loaded:", DF.shape)
display(DF.head())

Loaded: (32833, 57)


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,has_with,has_dont,has_remastered,has_version,has_like,has_your,has_original,has_remaster,has_down,has_this
0,6f807x0ima9a1j3VPbc7VN,i dont care with justin bieber loud luxury remix,ed sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,i dont care with justin bieber loud luxury remix,2019-06-14,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,1,0,0,0,0,0,0,0,0
1,0r7CVbZTWZgbTCYdfa2P31,memories dillon francis remix,maroon 5,67,63rPSO264uRjW1X5E6cWv6,memories dillon francis remix,2019-12-13,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
2,1z1Hg7Vb0AhHDiEmnDE79l,all the time don diablo remix,zara larsson,70,1HoSmj2eLcsrR0vE9gThr4,all the time don diablo remix,2019-07-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
3,75FpbthrwQmzHlBJLuGdC7,call you mine keanu silva remix,the chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,call you mine the remixes,2019-07-19,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
4,1e8PAfcKUYoKkxPhrHqw4x,someone you loved future humans remix,lewis capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,someone you loved future humans remix,2019-03-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_copy = DF.copy()
print("Working copy created:", df_copy.shape)

Working copy created: (32833, 57)


In [6]:
from sklearn.model_selection import train_test_split

# Work on the modeling copy
X = df_copy.drop(columns=["track_popularity"])
y = df_copy["track_popularity"]

# Step 1: Train (70%) vs Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=0.70, random_state=222, shuffle=True
)

# Step 2: split Temp into Dev (15%) and Test (15%)
X_dev, X_test, y_dev, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=222, shuffle=True
)

print(f"Train: {len(X_train)} ({len(X_train)/len(df_copy):.1%})")
print(f"Dev:   {len(X_dev)} ({len(X_dev)/len(df_copy):.1%})")
print(f"Test:  {len(X_test)} ({len(X_test)/len(df_copy):.1%})")

Train: 22983 (70.0%)
Dev:   4925 (15.0%)
Test:  4925 (15.0%)


In [8]:
cols_to_check = [
    "track_name", "track_artist", "track_album_name",
    "playlist_name", "playlist_subgenre"
]

total_count = len(X_train)

for col in [c for c in cols_to_check if c in X_train.columns]:
    unique_count = X_train[col].nunique(dropna=False)
    print(f"{col:25} → {unique_count:6} unique values ({unique_count/total_count:.1%} of total rows)")

track_name                →  17350 unique values (75.5% of total rows)
track_artist              →   8628 unique values (37.5% of total rows)
track_album_name          →  15011 unique values (65.3% of total rows)
playlist_name             →    423 unique values (1.8% of total rows)
playlist_subgenre         →     24 unique values (0.1% of total rows)


In [10]:
!pip install category_encoders

import pandas as pd
from category_encoders import LeaveOneOutEncoder

te_cols = ["track_artist", "playlist_name"]
count_cols = ["track_name", "track_artist", "track_album_name", "playlist_name"]

for c in count_cols:
    vc = X_train[c].value_counts()
    X_train[f"{c}_count"] = X_train[c].map(vc).fillna(0)
    X_dev[f"{c}_count"]   = X_dev[c].map(vc).fillna(0)
    X_test[f"{c}_count"]  = X_test[c].map(vc).fillna(0)

enc = LeaveOneOutEncoder(cols=te_cols, return_df=True, sigma=0.05)
X_train_te = enc.fit_transform(X_train[te_cols], y_train)
X_dev_te   = enc.transform(X_dev[te_cols])
X_test_te  = enc.transform(X_test[te_cols])

for c in te_cols:
    X_train[f"{c}_TE"] = X_train_te[c]
    X_dev[f"{c}_TE"]   = X_dev_te[c]
    X_test[f"{c}_TE"]  = X_test_te[c]
    X_train.drop(columns=[c], inplace=True)
    X_dev.drop(columns=[c], inplace=True)
    X_test.drop(columns=[c], inplace=True)

print("Done. Shapes:", X_train.shape, X_dev.shape, X_test.shape)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.5 MB/s eta 0:00:00
Done. Shapes: (22983, 60) (4925, 60) (4925, 60)


In [12]:
drop_cols = [
    "track_id", "playlist_id", "track_album_id",
    "track_name", "track_artist",
    "track_album_name", "playlist_name",
    "track_album_release_date"
]

X_train_model = X_train.drop(columns=drop_cols, errors="ignore").copy()
X_dev_model   = X_dev.drop(columns=drop_cols, errors="ignore").copy()
X_test_model  = X_test.drop(columns=drop_cols, errors="ignore").copy()

In [13]:
# Identify categorical columns left after TE
cat_cols = X_train_model.select_dtypes(include=["object", "category"]).columns
print("Categorical columns left:", list(cat_cols))

# Encode separately and align columns
X_train_enc = pd.get_dummies(X_train_model, columns=cat_cols, drop_first=True)
X_dev_enc   = pd.get_dummies(X_dev_model,   columns=cat_cols, drop_first=True)
X_test_enc  = pd.get_dummies(X_test_model,  columns=cat_cols, drop_first=True)

# Align columns so all sets match the train structure
X_dev_enc  = X_dev_enc.reindex(columns=X_train_enc.columns, fill_value=0)
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

print("Shapes after encoding:", X_train_enc.shape, X_dev_enc.shape, X_test_enc.shape)

Categorical columns left: ['playlist_genre', 'playlist_subgenre', 'key', 'mode', 'release_dow', 'release_dow_name', 'mode_label']
Shapes after encoding: (22983, 100) (4925, 100) (4925, 100)


In [16]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Train quick models on TRAIN only
lasso = Lasso(alpha=5, max_iter=10000, random_state=42).fit(X_train_enc, y_train)
ridge = Ridge(alpha=5, random_state=42).fit(X_train_enc, y_train)
gb    = GradientBoostingRegressor(random_state=42).fit(X_train_enc, y_train)
rf    = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1).fit(X_train_enc, y_train)

# Feature selection summary
lasso_sel = (np.abs(lasso.coef_) > 0).astype(int)
ridge_sel = (np.abs(ridge.coef_) > 0.05 * np.abs(ridge.coef_).max()).astype(int)
gb_sel    = (gb.feature_importances_ > gb.feature_importances_.mean()).astype(int)
rf_sel    = (rf.feature_importances_ > rf.feature_importances_.mean()).astype(int)

selection_df = pd.DataFrame({
    "Feature": X_train_enc.columns,
    "Lasso": lasso_sel,
    "Ridge": ridge_sel,
    "GradientBoost": gb_sel,
    "RandomForest": rf_sel
})

selection_df["Sum"] = selection_df[["Lasso","Ridge","GradientBoost","RandomForest"]].sum(axis=1)
selection_df = selection_df.sort_values("Sum", ascending=False)

print("\nTop 20 features by importance consensus:")
print(selection_df.head(20))


Top 20 features by importance consensus:
                  Feature  Lasso  Ridge  GradientBoost  RandomForest  Sum
45        track_artist_TE      1      1              1             1    4
46       playlist_name_TE      1      1              1             1    4
18       album_song_count      0      1              1             1    3
19    song_playlist_count      0      1              1             1    3
9            release_year      1      0              1             1    3
3             speechiness      0      1              0             1    2
12   instrumentalness_log      0      1              0             1    2
4            acousticness      0      1              0             1    2
11            release_day      1      0              0             1    2
8             duration_ms      1      0              0             1    2
1                  energy      0      1              0             1    2
5                liveness      0      1              0             1  

Selection Criteria
Features with ≥2 model votes (out of 4: Lasso, Ridge, Random Forest, Gradient Boosting) were selected for inclusion.
5 strong predictors identified by 3-4 models (high model agreement).
11 additional predictors identified by 2 models, showing consistent but moderate importance.
Total: 16 selected features retained for the modeling phase.

In [17]:
selected_features = selection_df.loc[selection_df["Sum"] >= 2, "Feature"].tolist()

# Create reduced matrices for modeling
X_train_sel = X_train_enc[selected_features].copy()
X_dev_sel   = X_dev_enc[selected_features].copy()
X_test_sel  = X_test_enc[selected_features].copy()

print(f"{len(selected_features)} features selected (Sum ≥ 2).")
print("Shapes:", X_train_sel.shape, X_dev_sel.shape, X_test_sel.shape)

print("\nSelected features preview:", selected_features)

16 features selected (Sum ≥ 2).
Shapes: (22983, 16) (4925, 16) (4925, 16)

Selected features preview: ['track_artist_TE', 'playlist_name_TE', 'album_song_count', 'song_playlist_count', 'release_year', 'speechiness', 'instrumentalness_log', 'acousticness', 'release_day', 'duration_ms', 'energy', 'liveness', 'artist_playlist_count', 'track_artist_count', 'valence', 'danceability']


In [18]:
!ls "/content/drive/MyDrive/Spotify_Project/"

1_Prep					  spotify_for_model.pkl
2_EDA					  spotify_max.xlsx
3_Cleansing				  spotify_min.xlsx
4_Feature_Engineering			  spotify_missing.xlsx
5_Data_Selection			  spotify_model_df_17feats.pkl
6_Models_Finetuning			  spotify_raw.parquet
outlier_summary.xlsx			  spotify_raw.pkl
spotify_clean_ready_2025-10-25_08-27.pkl  spotify_sweetviz.html
spotify_datatypes.xlsx			  spotify_unique.xlsx
spotify_EDA_full.pkl			  spotify_with_features.pkl
spotify_EDA.html


In [19]:
import pickle
import datetime

# Create a combined dict with selected feature sets
data_dict = {
    "X_train_sel": X_train_sel,
    "y_train": y_train,
    "X_dev_sel": X_dev_sel,
    "y_dev": y_dev,
    "X_test_sel": X_test_sel,
    "y_test": y_test,
    "selected_features": selected_features
}

# Define path with version 2
path = "/content/drive/MyDrive/Spotify_Project/5_Data_Selection/spotify_selected_features2.pkl"

# Save to pickle
with open(path, "wb") as f:
    pickle.dump(data_dict, f)

print(f"Saved: {path}")
print("Train:", X_train_sel.shape)
print("Dev:", X_dev_sel.shape)
print("Test:", X_test_sel.shape)
print("Features:", len(selected_features))


Saved: /content/drive/MyDrive/Spotify_Project/5_Data_Selection/spotify_selected_features2.pkl
Train: (22983, 16)
Dev: (4925, 16)
Test: (4925, 16)
Features: 16
